## **Benchmarking Tool for Thin-Film Transistors** ⚡

## **Student**:

Rodrigo Santos Batista – rsb6@cin.ufpe.br

CIn ­ UFPE

## **Research project**:

Benchmarking emerging technologies for near and far computing

inside sensors (BEeTle)

## **Coordinator**:

Stefan Michael Blawid – sblawid@cin.ufpe.br

CIn ­ UFPE

## **Using the Model** 💻

Below, insert the address of the `MODULE_OTFT` module that contains the files to be imported. This folder contains all the necessary files that the model uses to function. It can be loaded following the example: `sys.path.append('/content/drive/MyDrive/your/path/Model_OTFT')`, simply replace the path below with the corresponding path from your mounted drive containing the module.

In [1]:
# Load the used classes
from modules_otft._imports import *
from modules_otft._read_data import ReadData
from modules_otft._model import TFTModel
from modules_otft._grafics import  TFTGraphicsPlot
from modules_otft._menu import TFTMenu

from modules_otft._global_vars import *
from modules_otft._config import *
from modules_otft._utils import *


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------





---
## **Instantiating the used classes**
---


In [2]:
plot = TFTGraphicsPlot()
menu = TFTMenu()
read = ReadData()

---
## **Loading sample data** 📚


---



In [3]:
show_varibles()

| path: /home/rsb6/Desktop/Artigo JICS/Model_OTFT/datas/Fig1ab/tipo p
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_transfer: A
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| experimental_data_scale_output: uA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| current_typic: uA
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| loaded_voltages: -2, -50, -30